In [1]:
# pivot_table就是转换各个维度去观察数据, aggfunc就是在数据转换过程中的过程函数
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'student': ['小红', '小红', '李华', '李华', '小天', '小天'],
                    'class': ['001','001','001','001','002','002'],
                   'subject': ['C', 'Java', 'Python', 'C', 'C', 'Python'],
                   'grades': [80,  90, 78, 90, 80, 78]})

In [3]:
df

,student,class,subject,grades
0,小红,001,C,80
1,小红,001,Java,90
2,李华,001,Python,78
3,李华,001,C,90
4,小天,002,C,80
5,小天,002,Python,78


In [4]:
# pivot_table 语法
?df.pivot_table

In [5]:
# 获取每一门课程的平均分。
# subject , aggfunc="mean"
df.pivot_table(index="subject",aggfunc="mean")

,grades
subject,
C,83.333333
Java,90.000000
Python,78.000000


In [6]:
df.pivot_table(columns="subject")

subject,C,Java,Python
grades,83.333333,90.0,78.0


In [7]:
# 得出结论：
# index 作为纵轴索引
# columns 作为横轴索引
# 最终观察的时 value的值
# 为什么不是 student,class，而是 grades

In [8]:
df.dtypes

student    object
class      object
subject    object
grades      int64
dtype: object

In [9]:
df.pivot_table(index="subject",aggfunc=lambda x: type(x))

,class,grades,student
subject,,,
C,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>
Java,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>
Python,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>,<class 'pandas.core.frame.DataFrame'>


In [10]:
# tolist() 转换成列表
df.pivot_table(index="subject", aggfunc=lambda x: x.tolist())

,class,grades,student
subject,,,
C,"[001, 001, 002]","[80, 90, 80]","[小红, 李华, 小天]"
Java,[001],[90],[小红]
Python,"[001, 002]","[78, 78]","[李华, 小天]"


In [11]:
# 统计各班级的平均分？
df.pivot_table(index="class")

,grades
class,
001,84.5
002,79.0


In [12]:
# reset_index() 重置索引
df.pivot_table(index="class").reset_index()

,class,grades
0,001,84.5
1,002,79.0


In [13]:
# drop=True 删除class索引号
df.pivot_table(index="class").reset_index(drop=True)

,grades
0,84.5
1,79.0


In [14]:
# 统计各班级平均分及班级学生人数？
# df.student.nunique()      aggfunc=mean
# x.nunique() = df.class.nunique() = len(df.class.unique())
df.pivot_table(index="class", aggfunc={"grades": np.mean ,"student":lambda x:x.nunique()})

,grades,student
class,,
001,84.5,2
002,79.0,1


In [15]:
df.pivot_table(index="class", aggfunc={"grades": np.mean ,"student":lambda x:len(x.unique())})

,grades,student
class,,
001,84.5,2
002,79.0,1


In [18]:
# (3)统计各班级及各科目的平均分？
# class, subject
df.pivot_table(index="class", columns="subject")

grades             
subject      C  Java Python
class                      
001       85.0  90.0   78.0
002       80.0   NaN   78.0

In [20]:
# （4）统计各个班级(class)的各个科目(subject)的最高分(空值填充为0)
# fill_value 默认将空值填充为0 
df.pivot_table(index="class", columns="subject", values="grades",aggfunc=max, fill_value=0)

subject,C,Java,Python
class,,,
001,90,90,78
002,80,0,78


In [21]:
# (5) 统计各个班级的各个科目人数（空值填充0）
df.pivot_table(index="class", columns="subject", values="grades",aggfunc='count', fill_value=0)



subject,C,Java,Python
class,,,
001,2,1,1
002,1,0,1


In [23]:
# (6) 统计各个学生的最高分，最低分，平均分
df.pivot_table(index="student",values="grades",aggfunc=[max,min,np.mean])

,max,min,mean
,grades,grades,grades
student,,,
小天,80,78,79
小红,90,80,85
李华,90,78,84


In [27]:
# (7) 统计每个学生的所有科目最高分，最低分，平均分
df.pivot_table(index="student",columns="subject", values="grades",aggfunc=[max,min,np.mean],fill_value=0)


max             min             mean            
subject   C Java Python   C Java Python    C Java Python
student                                                 
小天       80    0     78  80    0     78   80    0     78
小红       80   90      0  80   90      0   80   90      0
李华       90    0     78  90    0     78   90    0     78

In [28]:
# stack() 行转列
df.pivot_table(index="student",values="grades",aggfunc=[max,min,np.mean]).stack()

,,max,min,mean
student,,,,
小天,grades,80,78,79
小红,grades,90,80,85
李华,grades,90,78,84


In [31]:
# reset_index() 表示索引列显示的位置
df.pivot_table(index="student",values="grades",aggfunc=[max,min,np.mean]).stack().reset_index(level=1,drop=True)

,max,min,mean
student,,,
小天,80,78,79
小红,90,80,85
李华,90,78,84
